In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2
import os
from dotenv import load_dotenv

In [2]:
# Connect to the database
load_dotenv()
conn = psycopg2.connect(
	dbname=os.getenv("DATABASE"),
	user=os.getenv("USER"),
	password=os.getenv("PASSWORD"),
	host=os.getenv("HOST")
)

cursor = conn.cursor()
cursor.execute("SELECT version();")

db_version = cursor.fetchone()
print("PostgreSQL version:", db_version)

PostgreSQL version: ('PostgreSQL 15.8 (Debian 15.8-0+deb12u1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit',)


# From points with aws data, split them to month and export to csv

In [ ]:
months = [4, 10]

# get the data for the months
for month in months:
    output_dir = os.getenv("DATAPATH") + f"{month}/"
    query = f"select * from weather_data where extract(month from date) = {month} and aws is not null"
    data = pd.read_sql_query(query, conn)
    data.to_csv(f"{output_dir}weather_data_{month}.csv", index=False)
    data.head()
    data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d %H:%M:%S')
    # Train data
    train_data = data[(data['date'] >= f'2019-{month}-05 00:00:00') & (data['date'] < f'2020-{month}-28')].reset_index(drop=True)
    train_data.to_csv(f"{output_dir}train_data_{month}.csv", index=False)
    # Test data
    test_data = data[(data['date'] < f'2019-{month}-05') | (data['date'] >= f'2020-{month}-28 00:00:00')].reset_index(drop=True)
    test_data.to_csv(f"{output_dir}test_data_{month}.csv", index=False)

/tmp/ipykernel_54445/3663735210.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(query, conn)
/tmp/ipykernel_54445/3663735210.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(query, conn)
